# 燈控器

## 使用Modbus控制燈控器

![img](lc103h_modbus_do.gif)

In [ ]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time
from struct import *


comport = 'COM7' # for linux: '/dev/ttyUSB0'
baudrate = 9600
databit = 8
parity = 'N'
stopbit = 1
timeout = 100 # ms

mb_port = serial.Serial(port=comport, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(timeout/1000.0)

mb_id = 1
addr = 0
value = 1

try:
    rr = master.execute(mb_id, cst.WRITE_SINGLE_COIL, addr, output_value=value)
    print("Write(addr, value)=%s" %(str(rr)))

except Exception as e:#Exception, e:
    print("modbus test Error: " + str(e))

master._do_close()


----
## 使用MQTT控制電燈

In [ ]:
# push mqtt msg

import paho.mqtt.client as mqtt
import time, json, datetime, sys, os
import random

def on_connect(client, userdata, flags, rc):
    m="Connected flags"+str(flags)+", result code "+str(rc)+", client_id  "+str(client)
    print(m)

# some online free broker:
#   iot.eclipse.org
#   test.mosquitto.org
#   broker.hivemq.com
broker_address = "broker.hivemq.com"
broker_port = 1883

client1 = mqtt.Client()    #create new instance
client1.on_connect = on_connect        #attach function to callback

time.sleep(0.5)
client1.connect(host=broker_address, port=broker_port, keepalive=60)      #connect to broker
topic_str = '/malo/icshop/light'
        
client1.loop_start()    #start the loop
time.sleep(0.5)
print("loop start")

#-- start to push data
for k in range(1, 7):
    value = k%2
    print('--> push', topic_str, value)
    client1.publish(topic_str, value, qos=1)
    time.sleep(5)

time.sleep(0.1)

client1.disconnect()
